In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [2]:
!pip install wandb datasets transformers sacrebleu==1.5.1  bert-score -qqq

     |████████████████████████████████| 1.7 MB 3.5 MB/s 
     |████████████████████████████████| 287 kB 49.8 MB/s 
     |████████████████████████████████| 2.9 MB 22.5 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 59 kB 7.1 MB/s 
     |████████████████████████████████| 139 kB 49.5 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 180 kB 48.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 125 kB 47.6 MB/s 
     |████████████████████████████████| 243 kB 50.8 MB/s 
     |████████████████████████████████| 1.3 MB 39.0 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 41.4 MB/s 
     |████████████████████████████████| 3.3 MB 47.3 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 271 kB 45.8 MB/s 
     |███████████████████

In [1]:
!nvidia-smi

Mon Oct 18 23:57:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import datetime
from tqdm import tqdm
model_checkpoint = 't5-small'
fp16 = True
todaydate = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
gdir = f'drive/My Drive/Colab Notebooks/{todaydate}/'
model_name=f'sparql-qald9-{model_checkpoint}-{todaydate}'
model_path='./models/'+model_name
ds_path= 'qald-text-to-sparql'
print(model_name)

sparql-qald9-t5-small-2021-10-19_00-01


In [4]:
# Flexible integration for any Python script
import wandb

# 1. Start a W&B run
wandb.init(project='text-to-sparql', entity='shahriar', name= model_name)

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
model_name

'sparql-qald9-t5-small-2021-10-19_00-01'

In [6]:
!mkdir models

In [ ]:
import torch
torch.cuda.empty_cache()

In [14]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, T5TokenizerFast
model = AutoModelForSeq2SeqLM.from_pretrained("yazdipour/text-to-sparql-t5-small-2021-10-18_23-00").to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer = T5TokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [8]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [9]:
!apt install git-lfs -qqq
!git config --global credential.helper store

Selecting previously unselected package git-lfs.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [7]:
!unzip qald-text-to-sparql.zip

Archive:  qald-text-to-sparql.zip
 extracting: qald-text-to-sparql/dataset_dict.json  
  inflating: qald-text-to-sparql/test.csv  
   creating: qald-text-to-sparql/test/
  inflating: qald-text-to-sparql/test/dataset.arrow  
  inflating: qald-text-to-sparql/test/dataset_info.json  
  inflating: qald-text-to-sparql/test/state.json  
  inflating: qald-text-to-sparql/train.csv  
   creating: qald-text-to-sparql/train/
  inflating: qald-text-to-sparql/train/dataset.arrow  
  inflating: qald-text-to-sparql/train/dataset_info.json  
  inflating: qald-text-to-sparql/train/state.json  


In [10]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
raw_datasets = load_from_disk(ds_path)
# raw_datasets = load_dataset("yazdipour/text-to-sparql-t5-lc-quad-v2", data_files={"train": "train.csv", "test": "test.csv"})

In [ ]:
# Preprocessing

In [15]:
print(raw_datasets['test']['translation'][0]['sparql'],'\n',
      raw_datasets['test']['translation'][0]['en'],'\n',
      tokenizer(raw_datasets['test']['translation'][0]['en']))

select distinct ?uri where [ res:Salt_Lake_City onto:timeZone ?uri ] 
 what is the time zone of salt lake city? 
 {'input_ids': [125, 19, 8, 97, 2901, 13, 3136, 6957, 690, 58, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [12]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry
print('\n',max_input_length, max_target_length)

100%|██████████| 408/408 [00:00<00:00, 723583.95it/s]


 85 299


In [13]:
source_lang = "en"
target_lang = "sparql"
prefix = "translate english to sparql2: "

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [23]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 150
    })
})

In [19]:
# Fine-tuning the model

In [17]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=fp16,
    push_to_hub=True
)

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]
    return preds, labels

In [24]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

In [20]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [21]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    P, R, F1 = scorer.score(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    gen_len = np.mean(prediction_lens)
    
    return {"gen_len":gen_len, 'P':P.mean(), 'R':R.mean(), 'F1':F1.mean(), "bleu-score": result["score"], "bleu-precisions": result["precisions"], "bleu-bp": result["bp"]}

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/yazdipour/sparql-qald9-t5-small-2021-10-19_00-01 into local empty directory.
Using amp fp16 backend


In [25]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 408
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 51
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Gen Len,P,R,F1,Bleu-score,Bleu-precisions,Bleu-bp
1,No log,2.405783,19.000000,0.394572,0.066015,0.225274,9.843835,"[72.36042012161415, 47.920433996383366, 33.929754804506295, 26.416482707873435]",0.234447


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Trainer is attempting to log a value of "[72.36042012161415, 47.920433996383366, 33.929754804506295, 26.416482707873435]" of type <class 'list'> for key "eval/bleu-precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=51, training_loss=2.07185333850337, metrics={'train_runtime': 35.8942, 'train_samples_per_second': 11.367, 'train_steps_per_second': 1.421, 'total_flos': 11639394926592.0, 'train_loss': 2.07185333850337, 'epoch': 1.0})

In [26]:
trainer.save_model(model_path)

Saving model checkpoint to ./models/sparql-qald9-t5-small-2021-10-19_00-01
Configuration saved in ./models/sparql-qald9-t5-small-2021-10-19_00-01/config.json
Model weights saved in ./models/sparql-qald9-t5-small-2021-10-19_00-01/pytorch_model.bin
tokenizer config file saved in ./models/sparql-qald9-t5-small-2021-10-19_00-01/tokenizer_config.json
Special tokens file saved in ./models/sparql-qald9-t5-small-2021-10-19_00-01/special_tokens_map.json


In [27]:
model_name

'sparql-qald9-t5-small-2021-10-19_00-01'

In [28]:
trainer.push_to_hub()

Saving model checkpoint to sparql-qald9-t5-small-2021-10-19_00-01
Configuration saved in sparql-qald9-t5-small-2021-10-19_00-01/config.json
Model weights saved in sparql-qald9-t5-small-2021-10-19_00-01/pytorch_model.bin
tokenizer config file saved in sparql-qald9-t5-small-2021-10-19_00-01/tokenizer_config.json
Special tokens file saved in sparql-qald9-t5-small-2021-10-19_00-01/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.36k/231M [00:00<?, ?B/s]

Upload file runs/Oct19_00-07-04_23829e71e870/events.out.tfevents.1634602139.23829e71e870.78.0:  68%|######8   …

Upload file runs/Oct19_00-07-04_23829e71e870/1634602139.9677083/events.out.tfevents.1634602139.23829e71e870.78…

Upload file training_args.bin: 100%|##########| 2.92k/2.92k [00:00<?, ?B/s]

To https://huggingface.co/yazdipour/sparql-qald9-t5-small-2021-10-19_00-01
   ebc8519..779e36b  main -> main

Dropping the following result as it does not have all the necessary field:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/yazdipour/sparql-qald9-t5-small-2021-10-19_00-01
   779e36b..c891ac2  main -> main



'https://huggingface.co/yazdipour/sparql-qald9-t5-small-2021-10-19_00-01/commit/779e36ba8e7bd6814435c0601af224c94ba9f6e3'

In [29]:
translator = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=0 #0 for cuda, -1 for cpu
)

In [30]:
translate= lambda q: (translator(prefix+q, max_length=100)[0]['translation_text'])

In [31]:
translate('How old is Bill Gates?')

'select distinct ?uri where [ ?uri dbo:date_of_death ?uri ]'

In [32]:
translate('Are Taiko some kind of Japanese musical instrument?  ')

'ask where [ dbo:taiko dbo:musical_instrument dbo:type dbo:Japan ]'